<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/mttr_interactive_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTTR - Interactive Demo
![visitors](https://visitor-badge.glitch.me/badge?page_id=mttr_collab)

This notebook provides a (limited) hands-on demonstration of MTTR (**M**ultimodal **T**racking **Tr**ansformer), originally introduced in our paper:

<div align="center">
<h3>
<b>
End-to-End Referring Video Object Segmentation with Multimodal Transformers
</b>
</h3>
</div>
<br>

Given a text query and a short clip based on a YouTube video, we demonstrate how MTTR can be used to segment the referred object instance throughout the video.

**Note** - for best performance make sure that GPU acceleration is turned on under: Runtime->Change runtime type->Hardware accelerator->GPU.
<br><br>

### Disclaimer
This is a **limited** demonstration of MTTR's performance. The model used here was trained **exclusively** on Refer-YouTube-VOS with window size `w=12` (as described in our paper). No additional training data was used whatsoever. 
Hence, the model's performance may be limited, especially on instances from unseen categories.

Additionally, slow processing times may be encountered, depending on the input clip length and/or resolution, and due to Colab's limited computational resources.

Finally, we emphasize that this demonstration is intended to be used for academic purposes only. We do not take any responsibility for how the created content is used or distributed, and discourage the users from copyright infringment of YouTube videos. <br><br>

And now, with all formalities aside, let's begin! 


## Dependency Installation

In [ ]:
%%capture
!pip install av moviepy gdown yt-dlp ruamel.yaml einops timm transformers
import torch
import torchvision
import torchvision.transforms.functional as F
from einops import rearrange
import numpy as np
from PIL import Image, ImageDraw, ImageOps, ImageFont
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from IPython.display import HTML
from base64 import b64encode
from tqdm.notebook import trange, tqdm
from transformers import logging
logging.set_verbosity_error()

## 1. Input Selection

First, let's select our input.

The following cell contains several YouTube examples for you to choose from. These samples were selected as they comply with the data distribution the model was trained on (i.e. they contain instances from categories that are present in Refer-YouTube-VOS).

Each sample metadata must contain 3 things:

*   the video's url
*   start/end points of the wanted sub-clip (in seconds)
*   the text queries, each one referring to a different object instance in the clip

To reduce the processing time we limit the sub-clip to be up to 10 seconds long and allow up to 2 input text queries.

Finally, you may also try feeding MTTR with your own input, but, as mentioned earlier, **keep in mind that performance may be limited**.

In [ ]:
# Choose (by un-commenting) one of the following:
video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=YThX7_8I3m0', (233, 243), ['guy in black performing tricks on a bike', 'a black bike used to perform tricks']
# video_url, (start_pt, end_pt), text_queries = 'https://www.youtube.com/watch?v=hwLo7aU1Aas', (1144, 1152), ['a man riding a surfboard', 'a black and white surfboard']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=yvJDHbrumak', (48, 55), ['a red ball thrown in the air', 'a black horse playing with a person']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=L-Wd4A8ESyk', (289, 297), ['a guy performing tricks on a skateboard', 'a black skateboard']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=4iTiRvk4FHY', (24, 34), ['man in red shirt playing tennis', 'white tennis racket held by a man in a red shirt']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=ZHwlmvuW4NY', (115, 125), ['white dog playing', 'brown and black dog playing']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=YThX7_8I3m0', (67, 77), ['guy in white shirt performing tricks on a bike', 'a black bike used to perform tricks']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=C7TCH927--g', (3, 13), ['a dog to the right', 'a cat to the left']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=0Z_WAF1GKfk', (143.5, 147.5), ['a dog to the left playing with a toy', 'a dog to the right playing with a toy']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=aEJJmebTLEs', (70, 80), ['a person hugging a dog', 'a white dog sitting']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=8sDF8lflCTs' ,(15, 23), ['person in blue riding a bike']
# video_url, (start_pt, end_pt), text_queries =  'https://www.youtube.com/watch?v=dQw4w9WgXcQ', (2.5, 7.5), ['a person dancing']


#OR - try your using own input in the following format: (but keep in mind that performance may be limited!)
# video_url, (start_pt, end_pt), text_queries =  f'https://www.youtube.com/watch?v=???????' ,(start_pnt, end_pnt), ['text query 1', 'text query 2']

assert  0 < end_pt - start_pt <= 10, 'error - the subclip length must be 0-10 seconds long'
assert  1 <= len(text_queries) <= 2, 'error - 1-2 input text queries are expected'

Next, let's download the chosen video and visualize the part that we are interested in.

Note that the video's resolution is limited to 360p.
This is so to reduce the processing time due to Colab's limited computational resources.

In [ ]:
download_resolution = 360
full_video_path = 'full_video.mp4'
input_clip_path = 'input_clip.mp4'

# download parameters:
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
# download the whole video:
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# extract the relevant subclip:
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)
    
# visualize the input clip:
input_clip = open(input_clip_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(input_clip).decode()
HTML("""<video width=720 controls><source src="%s" type="video/mp4"></video>""" % data_url)

## 2. Inference

Now that we have selected our input, let's initialize the pre-trained MTTR model ([PyTorch Hub](https://pytorch.org/hub/) is used here for simplicity):

In [ ]:
model, postprocessor = torch.hub.load('mttr2021/MTTR:main','mttr_refer_youtube_vos', force_reload=True)
model = model.cuda()

A few necessary auxiliaries:

In [ ]:
class NestedTensor(object):
    def __init__(self, tensors, mask):
        self.tensors = tensors
        self.mask = mask

def nested_tensor_from_videos_list(videos_list):
    def _max_by_axis(the_list):
      maxes = the_list[0]
      for sublist in the_list[1:]:
          for index, item in enumerate(sublist):
              maxes[index] = max(maxes[index], item)
      return maxes

    max_size = _max_by_axis([list(img.shape) for img in videos_list])
    padded_batch_shape = [len(videos_list)] + max_size
    b, t, c, h, w = padded_batch_shape
    dtype = videos_list[0].dtype
    device = videos_list[0].device
    padded_videos = torch.zeros(padded_batch_shape, dtype=dtype, device=device)
    videos_pad_masks = torch.ones((b, t, h, w), dtype=torch.bool, device=device)
    for vid_frames, pad_vid_frames, vid_pad_m in zip(videos_list, padded_videos, videos_pad_masks):
        pad_vid_frames[:vid_frames.shape[0], :, :vid_frames.shape[2], :vid_frames.shape[3]].copy_(vid_frames)
        vid_pad_m[:vid_frames.shape[0], :vid_frames.shape[2], :vid_frames.shape[3]] = False
    return NestedTensor(padded_videos.transpose(0, 1), videos_pad_masks.transpose(0, 1))

def apply_mask(image, mask, color, transparency=0.7):
    mask = mask[..., np.newaxis].repeat(repeats=3, axis=2)
    mask = mask * transparency
    color_matrix = np.ones(image.shape, dtype=np.float) * color
    out_image = color_matrix * mask + image * (1.0 - mask)
    return out_image

Next, let's use MTTR to process the input clip and text queries and generate the corresponding instance segmentation masks:

In [ ]:
window_length = 24  # length of window during inference
window_overlap = 6  # overlap (in frames) between consecutive windows

with torch.inference_mode():
  # read and preprocess the video clip:
  video, audio, meta = torchvision.io.read_video(filename=input_clip_path)
  video = rearrange(video, 't h w c -> t c h w')
  input_video = F.resize(video, size=360, max_size=640).cuda()
  input_video = input_video.to(torch.float).div_(255)
  input_video = F.normalize(input_video, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  video_metadata = {'resized_frame_size': input_video.shape[-2:], 'original_frame_size': video.shape[-2:]}
  
  # partition the clip into overlapping windows of frames:
  windows = [input_video[i:i+window_length] for i in range(0, len(input_video), window_length - window_overlap)]
  # clean up the text queries:
  text_queries = [" ".join(q.lower().split()) for q in text_queries]

  pred_masks_per_query = []
  t, _, h, w = video.shape
  for text_query in tqdm(text_queries, desc='text queries'):
    pred_masks = torch.zeros(size=(t, 1, h, w))
    for i, window in enumerate(tqdm(windows, desc='windows')):
      window = nested_tensor_from_videos_list([window])
      valid_indices = torch.arange(len(window.tensors)).cuda()
      outputs = model(window, valid_indices, [text_query])
      window_masks = postprocessor(outputs, [video_metadata], window.tensors.shape[-2:])[0]['pred_masks']
      win_start_idx = i*(window_length-window_overlap)
      pred_masks[win_start_idx:win_start_idx + window_length] = window_masks
    pred_masks_per_query.append(pred_masks)

Finally, we apply the generated instance masks and their corresponding text queries on the input clip for visualization:

In [ ]:
# RGB colors for instance masks:
light_blue = (41, 171, 226)
purple = (237, 30, 121)
dark_green = (35, 161, 90)
orange = (255, 148, 59)
colors = np.array([light_blue, purple, dark_green, orange])

# width (in pixels) of the black strip above the video on which the text queries will be displayed:
text_border_height_per_query = 36

video_np = rearrange(video, 't c h w -> t h w c').numpy() / 255.0
# del video
pred_masks_per_frame = rearrange(torch.stack(pred_masks_per_query), 'q t 1 h w -> t q h w').numpy()
masked_video = []
for vid_frame, frame_masks in tqdm(zip(video_np, pred_masks_per_frame), total=len(video_np), desc='applying masks...'):
  # apply the masks:
  for inst_mask, color in zip(frame_masks, colors):
    vid_frame = apply_mask(vid_frame, inst_mask, color / 255.0)
  vid_frame = Image.fromarray((vid_frame * 255).astype(np.uint8))
  # visualize the text queries:
  vid_frame = ImageOps.expand(vid_frame, border=(0, len(text_queries)*text_border_height_per_query, 0, 0))
  W, H = vid_frame.size
  draw = ImageDraw.Draw(vid_frame)
  font = ImageFont.truetype(font='LiberationSans-Regular.ttf', size=30)
  for i, (text_query, color) in enumerate(zip(text_queries, colors), start=1):
      w, h = draw.textsize(text_query, font=font)
      draw.text(((W - w) / 2, (text_border_height_per_query * i) - h - 3),
                text_query, fill=tuple(color) + (255,), font=font)
  masked_video.append(np.array(vid_frame))

# generate and save the output clip:
output_clip_path = 'output_clip.mp4'
clip = ImageSequenceClip(sequence=masked_video, fps=meta['video_fps'])
clip = clip.set_audio(AudioFileClip(input_clip_path))
clip.write_videofile(output_clip_path, fps=meta['video_fps'], audio=True)
del masked_video

# visualize the output clip:
output_clip = open(output_clip_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(output_clip).decode()
HTML("""<video width=720 controls><source src="%s" type="video/mp4"></video>""" % data_url)